In [1]:
from diffusers import AutoencoderKL
import torch
import os
import numpy as np
import librosa
from tqdm import tqdm
import soundfile as sf
import torchaudio
import json
from IPython.display import Audio
import matplotlib.pyplot as plt

In [3]:
ac_train_json = "/home/v-yuancwang/AUDIT_v2/medata_infos/ac_train.json"
with open(ac_train_json, "r") as f:
    ac_train_list = json.load(f)
ac_train_list[:2]

[{'mel': '/blob/v-yuancwang/DiffAudioImg/VGGSound/data/ac_train/mel/---1_cCGK4M_0_10000.npy',
  'caption': 'Idling car, train blows horn and passes'},
 {'mel': '/blob/v-yuancwang/DiffAudioImg/VGGSound/data/ac_train/mel/---lTs1dxhU_30000_40000.npy',
  'caption': 'A racing vehicle engine is heard passing by'}]

In [ ]:
vae = AutoencoderKL.from_pretrained("/blob/v-yuancwang/AudioEditingModel/VAE_GAN/checkpoint-40000/vae")
vae.requires_grad_(False)
torch_device = torch.device("cuda:1")
vae.to(torch_device)

In [15]:
vae_lists = []

for info in tqdm(ac_train_list[:10]):
    mel_path = info["mel"]
    wav_path = mel_path.replace("/mel/", "/wav/").replace(".npy", ".wav")
    wav, sr = librosa.load(wav_path, sr=16000)
    if len(wav) < 159500:
        continue
    mel = np.load(mel_path)

    test_mel_tensor = np.expand_dims(np.expand_dims(mel[:624], 0), 0)
    test_mel_tensor = torch.Tensor(test_mel_tensor).to(torch_device)
    with torch.no_grad():
        posterior = vae.encode(test_mel_tensor).latent_dist
        z = posterior.sample()
        vae_output = vae.decode(z).sample
    vae_res = vae_output[0][0].cpu().numpy()

    np.save(mel_path.replace("/mel/", "/vae_mel/"), vae_res)
    vae_lists.append(info)

with open("/home/v-yuancwang/AUDIT_v2/hifigan_ft_infos/ac_train.json", "w") as f:



100%|██████████| 10/10 [00:01<00:00,  5.74it/s]
